In [57]:
SEASON = '1980-81'

In [58]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['zTotals'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK'])
p_df['zWS'] = zscore(p_df['WS'])
p_df['zVORP'] = zscore(p_df['VORP'])
p_df['zBPM'] = zscore(p_df['BPM'])
p_df['zTmsWins'] = zscore(p_df['TmsWins'])
p_df['RawScore'] = p_df['zTotals'] + p_df['zWS'] + p_df['zVORP'] + p_df['zBPM'] + p_df['zTmsWins']
p_df['PlayerScore'] = p_df['RawScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,DBPM,BPM,VORP,TmsWins,zTotals,zWS,zVORP,zBPM,zTmsWins,RawScore
1,Julius Erving,PHI,SF,104.827897,30,82,82.0,35.0,9.7,18.6,...,2.3,7.5,6.9,62.0,3.656077,3.203694,4.315007,3.399472,1.658060,16.232310
2,Kareem Abdul-Jabbar,LAL,C,96.364725,33,80,NaN,37.2,10.5,18.2,...,1.4,6.1,6.1,54.0,4.114764,3.369841,3.724943,2.790123,1.019330,15.019000
3,Larry Bird,BOS,PF,79.574995,24,82,82.0,39.5,8.8,18.3,...,1.8,4.3,5.2,62.0,3.567869,2.206816,3.061122,2.006675,1.658060,12.500540
4,Moses Malone,HOU,C,75.262219,25,80,NaN,40.6,10.1,19.3,...,-1.2,3.3,4.3,40.0,4.785151,3.170465,2.397300,1.571426,-0.098447,11.825895
5,Marques Johnson,MIL,SF,74.064857,24,76,NaN,33.4,8.4,15.2,...,0.5,5.6,4.9,60.0,2.385870,2.339733,2.839848,2.572499,1.498377,11.636326
6,Adrian Dantley,UTA,SF,71.766999,25,80,NaN,42.7,11.4,20.3,...,-1.4,4.3,5.4,28.0,3.973629,3.137236,3.208637,2.006675,-1.056541,11.269636
7,Robert Parish,BOS,C,71.236117,27,82,78.0,28.0,7.7,14.2,...,2.1,5.4,4.3,62.0,2.403512,2.240045,2.397300,2.485449,1.658060,11.184365
8,Artis Gilmore,CHI,C,63.027626,31,82,NaN,34.5,6.7,10.0,...,1.2,4.3,4.5,45.0,2.280019,2.705255,2.544816,2.006675,0.300759,9.837524
9,George Gervin,SAS,SG,59.112441,28,82,NaN,33.7,10.4,21.1,...,-1.2,2.9,3.5,52.0,3.003332,2.107128,1.807237,1.397326,0.859648,9.174670
10,Cedric Maxwell,BOS,SF,56.251744,25,81,81.0,33.7,5.4,9.3,...,0.7,3.5,3.8,62.0,1.062737,2.273274,2.028510,1.658476,1.658060,8.681057
